In [1]:
# Imports and environment setup
import os
import requests
import pandas as pd
import numpy as np
import datetime as dt
import dotenv
dotenv.load_dotenv(dotenv.find_dotenv(filename=".env"))

# Dune
import dune_client
from dune_client.client import DuneClient
from dune_client.types import QueryParameter
from dune_client.query import QueryBase

# tsfresh & model
from tsfresh import extract_features, select_features
from tsfresh.utilities.dataframe_functions import roll_time_series
from tsfresh.feature_extraction import (
    MinimalFCParameters, 
    EfficientFCParameters, 
    ComprehensiveFCParameters
)
import xgboost as xgb


2025-09-20 14:47:59,226 INFO numba.cuda.cudadrv.driver init


In [6]:
# Key Variables
TIMEZONE = os.environ.get("TZ", "Europe/Madrid")
DAYS_BACK = 365
TARGET_COIN = "ethereum"
CG_TOP_N = 10

DUNE_QUERIES = {
    "economic_security": 1933076,
    "daily_dex_volume": 4388,
    "btc_etf_flows": 5795477,
    "eth_etf_flows": 5795645,
    "total_defi_users": 2972,
    "median_gas": 2981260,
}
DUNE_API_KEY = os.getenv("DUNE_API_KEY")
PREFER_DUNE_CSV = True
DUNE_CSV_PATH = "OutputData/Dune_Metrics.csv"

DERIBIT_CURRENCY = "BTC"      # or "ETH"
DERIBIT_RESOLUTION = "1D"
DERIBIT_DAYS = 365

TSFRESH_PRESET = "Comprehensive"   # "Minimal" | "Efficient" | "Comprehensive"
MAX_TIMESHIFT = 7
FDR_LEVEL = 0.10

XGB_PARAMS = {
    "max_depth": 5,
    "eta": 0.1,
    "min_child_weight": 3.0,
    "subsample": 0.9,
    "colsample_bytree": 0.9,
    "n_estimators": 600,
    "objective": "reg:squarederror",
    "tree_method": "hist",   # "gpu_hist" if GPU
    "eval_metric": "mae",
    "random_state": 42,
    "verbosity": 0,
}

os.makedirs("OutputData", exist_ok=True)

# CoinGecko headers
CG_API_KEY = os.getenv("COINGECKO_API_KEY")
CG_HEADERS = {
    "accept": "application/json",
    "x_cg_demo_api_key": CG_API_KEY
}


In [120]:
# API Wraps-> All helpers return a DataFrame with:
# - daily, tz-naive index named 'date'
# - lowercase column names (e.g., prices_eth, marketcaps_btc, total_volumes_eth, dvol_btc)

def cg_universe(n=CG_TOP_N):
    url = "https://api.coingecko.com/api/v3/coins/markets?vs_currency=usd"
    js = requests.get(url, headers=CG_HEADERS).json()
    df = pd.DataFrame(js)
    uni = list(df.head(n)["id"].values)
    if TARGET_COIN not in uni:
        uni = [TARGET_COIN] + [c for c in uni if c != TARGET_COIN]
    return uni

def cgpriceactiondaily(coins, days=DAYS_BACK):
    end = int(dt.datetime.now(dt.timezone.utc).timestamp()) * 1000
    start = int((dt.datetime.now(dt.timezone.utc) - dt.timedelta(days=days)).timestamp()) * 1000
    out = None
    for c in coins:
        try:
            url = f"https://api.coingecko.com/api/v3/coins/{c}/market_chart/range?vs_currency=usd&from={start}&to={end}"
            js = requests.get(url, headers=CG_HEADERS).json()
            p = pd.DataFrame(js["prices"],        columns=["t", f"prices_{c}"])
            m = pd.DataFrame(js["market_caps"],   columns=["t", f"marketcaps_{c}"])
            v = pd.DataFrame(js["total_volumes"], columns=["t", f"total_volumes_{c}"])
            df = p.merge(m, on="t").merge(v, on="t")
            df["t"] = pd.to_datetime(df["t"], unit="ms", utc=True)
            df = df.set_index("t")
            df.columns = [col.lower() for col in df.columns]
            df = _to_daily_naive(df)
            out = df if out is None else out.join(df, how="inner")
        except Exception:
            continue
    return out if out is not None else pd.DataFrame()

def deribit_dvol_daily(currency=DERIBIT_CURRENCY, days=DERIBIT_DAYS):
    end = int(dt.datetime.now().timestamp()) * 1000
    start = int((dt.datetime.now() - dt.timedelta(days=days)).timestamp()) * 1000
    js = requests.post(
        "https://www.deribit.com/api/v2/",
        json={"method": "public/get_volatility_index_data",
              "params": {"currency": currency, "resolution": "1D",
                         "end_timestamp": end, "start_timestamp": start}}
    ).json()
    d = pd.DataFrame(js["result"]["data"], columns=["t","open","high","low","dvol"])
    d["t"] = pd.to_datetime(d["t"], unit="ms", utc=True)
    df = d.set_index("t")[["dvol"]].rename(columns={"dvol": f"dvol_{currency.lower()}"})
    return _to_daily_naive(df)

def dune_metrics_daily(path=DUNE_CSV_PATH):
    if not os.path.exists(path):
        return pd.DataFrame()
    df = pd.read_csv(path, index_col="date", parse_dates=True)
    df.columns = [c.lower() for c in df.columns]
    return _to_daily_naive(df)

# Dune API Call Function: Not finalized-> need more credits
def fetch_dune_queries_df(query_ids):
    DUNE_API_KEY = os.environ.get("DUNE_API_KEY")
    dune = DuneClient(
        api_key= DUNE_API_KEY,
        request_timeout=300,
        base_url="https://api.dune.com"
    )
    count=0
    for queryid in query_ids:
        print(f"executing {queryid}")
        query = QueryBase(
            query_id=queryid
            # uncomment and change the parameter values if needed
            # params=[
            #     QueryParameter.text_type(name="contract", value="0x6B175474E89094C44Da98b954EedeAC495271d0F"), # default is DAI
            #     QueryParameter.text_type(name="owner", value="owner"), # default using vitalik.eth's wallet 
            # ],
        )
        query_result =dune.run_query_dataframe(
            query=query
            , ping_frequency = 2 # uncomment to change the seconds between checking execution status, default is 1 second
            # , performance="large" # uncomment to run query on large engine, default is medium
            , batch_size = 365 # uncomment to change the maximum number of rows to retrieve per batch of results, default is 32_000
        )
        for column in query_result.columns: 
            try: 
                query_result[column]= pd.to_datetime(query_result[column], UTC =True)
                query_result.rename({column: 'date'}, axis =1, inplace=True)
                query_result.set_index(column, inplace=True)
                break
            except: continue
        if count == 0: df= query_result
        else: df= query_result.join(df, how="inner")
    return query_result

In [ ]:
universe = cg_universe(CG_TOP_N)
queries = list(DUNE_QUERIES.values())
O = cgpriceactiondaily(universe, DAYS_BACK)
D = deribit_dvol_daily(DERIBIT_CURRENCY, DERIBIT_DAYS)
U = dune_metrics_daily(DUNE_CSV_PATH) 
# # Or for api calls: 
# U = fetch_dune_queries_df(queries)
OCA = O.join(D, how="left").join(U, how="left").dropna(how="any")
